In [3]:
import numpy as np
import pandas as pd
from shared.utils import load_data
from datasets import preprocess_dataset, datasets_types

from sklearn.cluster import KMeans
from scipy.stats import beta as beta_dist
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from art.attacks.evasion import HopSkipJump

from art.estimators.classification.scikitlearn import ScikitlearnClassifier
from art.utils import to_categorical
from art.attacks.evasion import ZooAttack
from art.estimators.classification import SklearnClassifier
seed = 42

In [ ]:
name = "CIC-IDS_2017_2"
df = load_data(
            [
                "./shared/data/CIC_2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
                "./shared/data/CIC_2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
                "./shared/data/CIC_2017/Tuesday-WorkingHours.pcap_ISCX.csv"
            ],
            seed
        )
print("Dataset loaded")
df_preprocessed = preprocess_dataset(
    df, save=True, dataset_type="CIC_2017", seed=seed, load=False, name_save=name, name_load=name)
print("Dataset preprocessed")

Dataset cargado
Loading new data
labels: {'Infiltration', 'SSH-Patator', 'Web Attack � XSS', 'FTP-Patator', 'Web Attack � Brute Force', 'Web Attack � Sql Injection'}
Dataset Preprocesado


In [7]:
x_train = df_preprocessed.x_train
y_train = df_preprocessed.y_train
x_test = df_preprocessed.x_test
y_test = df_preprocessed.y_test

y_train = y_train.apply(lambda x: int(x))
y_test = y_test.apply(lambda x: int(x))

print(y_train.unique())
print(y_test.unique())

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

[0 1]
[0 1]
(633413, 70)
(633413,)
(271464, 70)
(271464,)


In [8]:
# Train a RF Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
# Train a MLP Classifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(32),max_iter=200,verbose=False,batch_size=200,early_stopping=True,activation='tanh',solver='adam')
mlp.fit(x_train, y_train)

MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=32)

In [10]:
# Train a NB Classifier
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)

GaussianNB()

In [11]:
# Train a DT Classifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [12]:
# Train a LR Classifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [13]:
FUNCTIONAL_FEATURES = [
 ' min_seg_size_forward',' Bwd Header Length',' Destination Port'
 'Init_Win_bytes_forward',' Init_Win_bytes_backward',' Bwd Packets/s'
 'Total Length of Fwd Packets',' Subflow Fwd Bytes',' Max Packet Length'
 'Bwd Packet Length Max',' Avg Bwd Segment Size',' Bwd Packet Length Mean'
 ' Fwd Packet Length Max',' Average Packet Size',' Packet Length Std'
 ' Packet Length Mean',' Bwd Packet Length Std',' Bwd Packet Length Min'
 ' Fwd Packet Length Std',' Fwd Packet Length Min',' Min Packet Length'
 ' Fwd Packet Length Mean',' Avg Fwd Segment Size',' act_data_pkt_fwd'
 ' Total Fwd Packets','Subflow Fwd Packets',' Total Backward Packets']
print(len(FUNCTIONAL_FEATURES))
FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c not in FUNCTIONAL_FEATURES][:-1]
print(FUNCTIONAL_FEATURES_IDEXES, len(FUNCTIONAL_FEATURES_IDEXES))
NON_FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c in FUNCTIONAL_FEATURES]
print(NON_FUNCTIONAL_FEATURES_IDEXES, len(NON_FUNCTIONAL_FEATURES_IDEXES))
print(len(df.columns))

19
[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 53, 56, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68] 58
[3, 7, 13, 33, 50, 51, 52, 54, 55, 59, 61] 11
70


In [14]:
attacks_data = x_test[y_test == 1]
normal_data = x_test[y_test == 0]
print(attacks_data.shape, normal_data.shape)

(4815, 70) (266649, 70)


In [15]:
# Create ART classifier
class Classifier(ScikitlearnClassifier):
    
    def __init__(self, model, clip_values=None, preprocessing=(0, 1), attacks=[]):
        super(Classifier, self).__init__(model=model, clip_values=clip_values, preprocessing=preprocessing)
        self._attacks = attacks

    def predict(self, x, **kwargs):
        # Set attacks features to X
        for i in FUNCTIONAL_FEATURES_IDEXES:
            for j in range(len(x)):
                x[j][i] = self._attacks[j][i]
        predictions = self._model.predict(x)
        return to_categorical(self._model.predict(x)[0], nb_classes=self._get_nb_classes())

In [16]:
models = [rf, mlp, nb, dt, lr]
for model in models:
    # Create a SklearnClassifier wrapper around the trained classifier
    # classifier = Classifier(model=rf, clip_values=(0, 1), attacks=attacks_data)
    classifier = SklearnClassifier(model=model, clip_values=(0, 1))
    # Create the ZOO attack
    attack = ZooAttack(classifier=classifier, targeted=True, nb_parallel=60)
    # Generate adversarial data on the testing set
    x_test_adv = attack.generate(attacks_data[:500], np.zeros((attacks_data[:500].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:500], normal_data[:500]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:500].shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:500]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))
    # Evaluate the Model with the adversarial data and the non adversarial data
    print("====================> Model: ", model)
    print("---------- Non adversarial data")
    print(classification_report(non_adv_y_test, model.predict(non_adv_x_test)))
    print("Accuracy: ", accuracy_score(non_adv_y_test, model.predict(non_adv_x_test)))
    # Calculate the detection rate (attacks TP / attacks TP + attacks FN)
    print("Detection Rate: ", recall_score(non_adv_y_test, model.predict(non_adv_x_test)))
    print("F1 Score: ", f1_score(non_adv_y_test, model.predict(non_adv_x_test)))
    print("ROC AUC Score: ", roc_auc_score(non_adv_y_test, model.predict(non_adv_x_test)))
    print("---------- Adversarial data")
    print(classification_report(adv_y_test, model.predict(adv_x_test)))
    print("Accuracy: ", accuracy_score(adv_y_test, model.predict(adv_x_test)))
    print("Detection Rate: ", recall_score(adv_y_test, model.predict(adv_x_test)))
    print("F1 Score: ", f1_score(adv_y_test, model.predict(adv_x_test)))
    print("ROC AUC Score: ", roc_auc_score(adv_y_test, model.predict(adv_x_test)))

ZOO: 100%|██████████| 500/500 [00:14<00:00, 33.61it/s]


====================> Model:  RandomForestClassifier(random_state=42)
---------- Non adversarial data
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       500
         1.0       1.00      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       1.00      0.99      0.99      1000
weighted avg       1.00      0.99      0.99      1000

Accuracy:  0.995
Detection Rate:  0.99
F1 Score:  0.9949748743718593
ROC AUC Score:  0.995
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.57      1.00      0.72       500
         1.0       1.00      0.24      0.39       500

    accuracy                           0.62      1000
   macro avg       0.78      0.62      0.56      1000
weighted avg       0.78      0.62      0.56      1000

Accuracy:  0.62
Detection Rate:  0.24
F1 Score:  0.3870967741935484
ROC AUC Score:  0.62


ZOO: 100%|██████████| 500/500 [00:02<00:00, 233.34it/s]


====================> Model:  MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=32)
---------- Non adversarial data
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99       500
         1.0       1.00      0.97      0.98       500

    accuracy                           0.98      1000
   macro avg       0.99      0.98      0.98      1000
weighted avg       0.99      0.98      0.98      1000

Accuracy:  0.985
Detection Rate:  0.97
F1 Score:  0.9847715736040609
ROC AUC Score:  0.985
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71       500
         1.0       1.00      0.19      0.32       500

    accuracy                           0.60      1000
   macro avg       0.78      0.60      0.52      1000
weighted avg       0.78      0.60      0.52      1000

Accuracy:  0.596
Detection Rate:  0.192
F1 Score:  0.32214

ZOO: 100%|██████████| 500/500 [00:02<00:00, 227.20it/s]


====================> Model:  GaussianNB()
---------- Non adversarial data
              precision    recall  f1-score   support

         0.0       1.00      0.88      0.94       500
         1.0       0.90      1.00      0.94       500

    accuracy                           0.94      1000
   macro avg       0.95      0.94      0.94      1000
weighted avg       0.95      0.94      0.94      1000

Accuracy:  0.941
Detection Rate:  0.998
F1 Score:  0.9441816461684012
ROC AUC Score:  0.9410000000000001
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       1.00      0.88      0.94       500
         1.0       0.90      1.00      0.94       500

    accuracy                           0.94      1000
   macro avg       0.95      0.94      0.94      1000
weighted avg       0.95      0.94      0.94      1000

Accuracy:  0.941
Detection Rate:  0.998
F1 Score:  0.9441816461684012
ROC AUC Score:  0.9410000000000001


ZOO: 100%|██████████| 500/500 [00:01<00:00, 260.93it/s]


====================> Model:  DecisionTreeClassifier()
---------- Non adversarial data
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       500
         1.0       1.00      0.99      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.997
Detection Rate:  0.994
F1 Score:  0.9969909729187563
ROC AUC Score:  0.997
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.78       500
         1.0       1.00      0.45      0.62       500

    accuracy                           0.73      1000
   macro avg       0.82      0.73      0.70      1000
weighted avg       0.82      0.73      0.70      1000

Accuracy:  0.726
Detection Rate:  0.452
F1 Score:  0.6225895316804407
ROC AUC Score:  0.726


ZOO: 100%|██████████| 500/500 [00:03<00:00, 151.34it/s]

====================> Model:  LogisticRegression()
---------- Non adversarial data
              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80       500
         1.0       1.00      0.51      0.67       500

    accuracy                           0.75      1000
   macro avg       0.83      0.75      0.74      1000
weighted avg       0.83      0.75      0.74      1000

Accuracy:  0.753
Detection Rate:  0.508
F1 Score:  0.6728476821192053
ROC AUC Score:  0.753
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.54      1.00      0.70       500
         1.0       0.99      0.14      0.25       500

    accuracy                           0.57      1000
   macro avg       0.76      0.57      0.48      1000
weighted avg       0.76      0.57      0.48      1000

Accuracy:  0.571
Detection Rate:  0.144
F1 Score:  0.2513089005235602
ROC AUC Score:  0.571


In [17]:
class MultiArmedBanditThompsonSampling:

    def __init__(self, n_arms, n_clusters):
        self.n_arms = n_arms
        self.n_clusters = n_clusters
        self.arms = [RandomForestClassifier(), DecisionTreeClassifier(), GaussianNB()]
        self.cluster_centers = None
        self.cluster_assignments = None
        self.reward_sums = {}
        for cluster in range(n_clusters):
            self.reward_sums[cluster] = np.zeros(n_arms)
        self.alpha = np.ones(self.n_arms)
        self.beta = np.ones(self.n_arms)

    def train(self, X_train, y_train):
        kmeans = KMeans(n_clusters=self.n_clusters)
        self.cluster_assignments = kmeans.fit_predict(X_train)
        self.cluster_centers = kmeans.cluster_centers_
        # Print the number of samples in each cluster

        for i in range(self.n_clusters):
            print('Cluster {}: {}'.format(
                i, np.sum(self.cluster_assignments == i)))
            cluster_mask = self.cluster_assignments == i
            cluster_X_train = X_train[cluster_mask]
            cluster_y_train = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Training arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_train == arm
                arm_X_train = cluster_X_train[arm_mask]
                arm_y_train = cluster_y_train[arm_mask]
                if len(arm_X_train) > 0 and len(np.unique(arm_y_train)) > 1:
                    self.arms[arm].fit(arm_X_train, arm_y_train)
                else:
                    self.arms[arm].fit(X_train, y_train)

        # Set the arms rewards for each cluster
        for i in range(self.n_clusters):
            cluster_mask = self.cluster_assignments == i
            cluster_X_test = X_train[cluster_mask]
            cluster_y_test = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Setting reward_sums arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_test == arm
                arm_X_test = cluster_X_test[arm_mask]
                arm_y_test = cluster_y_test[arm_mask]
                if len(arm_X_test) > 0:
                    arm_y_pred = self.arms[arm].predict(arm_X_test)
                    self.reward_sums[i][arm] = np.mean(
                        arm_y_pred == arm_y_test)

    def select_arm(self, cluster):
        # Select the arm with the highest reward
        theta = np.zeros(self.n_arms)
        for arm in range(self.n_arms):
            theta[arm] = np.random.beta(self.alpha[arm] + self.reward_sums[cluster]
                                        [arm], self.beta[arm] + 1 - self.reward_sums[cluster][arm])
        return np.argmax(theta)

    def predict(self, X_test):
        # Select the arm for each sample
        arms = np.zeros(len(X_test))
        for i in range(len(X_test)):
            cluster = np.argmin(np.linalg.norm(
                self.cluster_centers - X_test[i], axis=1))
            arms[i] = self.select_arm(cluster)
        # Predict using the selected arm
        y_pred = np.zeros(len(X_test))
        for arm in range(self.n_arms):
            arm_mask = arms == arm
            arm_X_test = X_test[arm_mask]
            if len(arm_X_test) > 0:
                y_pred[arm_mask] = self.arms[arm].predict(arm_X_test)
        return y_pred, arms

In [18]:
# Train the MAB
mab = MultiArmedBanditThompsonSampling(n_arms=3, n_clusters=2)
mab.train(x_train, y_train)

d:\anaconda3\envs\menv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\anaconda3\envs\menv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "d:\anaconda3\envs\menv\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "d:\anaconda3\envs\menv\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "d:\anaconda3\envs\menv\lib\subprocess.py", line 1436, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


Cluster 0: 480204
Training arm 0 on cluster 0
Training arm 1 on cluster 0
Training arm 2 on cluster 0
Cluster 1: 153209
Training arm 0 on cluster 1
Training arm 1 on cluster 1
Training arm 2 on cluster 1
Setting reward_sums arm 0 on cluster 0
Setting reward_sums arm 1 on cluster 0
Setting reward_sums arm 2 on cluster 0
Setting reward_sums arm 0 on cluster 1
Setting reward_sums arm 1 on cluster 1
Setting reward_sums arm 2 on cluster 1


In [19]:
# Create a SklearnClassifier wrapper around the trained classifier
#classifier = Classifier(model=rf, clip_values=(0, 1), attacks=attacks_data)
models = [rf, mlp, nb, dt, lr]
accuracys = []
drs = []
f1s = []
rocs = []
for model in models:
    classifier = SklearnClassifier(model=model, clip_values=(0, 1))
    # Create the ZOO attack
    attack = ZooAttack(classifier=classifier, targeted=True, nb_parallel=60)
    # Generate adversarial data on the testing set
    x_test_adv = attack.generate(attacks_data[:500], np.zeros((attacks_data[:500].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:500], normal_data[:500]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:500].shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:500]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))


    # Evaluate the Model with the adversarial data and the non adversarial data
    print("====================> Model: ", model)
    # print("---------- Non adversarial data")
    # print(classification_report(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("Accuracy: ", accuracy_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # # Calculate the detection rate (attacks TP / attacks TP + attacks FN)
    # print("Detection Rate: ", recall_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("F1 Score: ", f1_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("ROC AUC Score: ", roc_auc_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    print("---------- Adversarial data")
    accuracy = accuracy_score(adv_y_test, mab.predict(adv_x_test)[0])
    accuracys.append(accuracy)
    dr = recall_score(adv_y_test, mab.predict(adv_x_test)[0])
    drs.append(dr)
    f1 = f1_score(adv_y_test, mab.predict(adv_x_test)[0])
    f1s.append(f1)
    roc = roc_auc_score(adv_y_test, mab.predict(adv_x_test)[0])
    rocs.append(roc)
    print(classification_report(adv_y_test, mab.predict(adv_x_test)[0]))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", dr)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

print("=====================================")
print("Accuracy: ", sum(accuracys)/len(accuracys))
print("Detection Rate: ", sum(drs)/len(drs))
print("F1 Score: ", sum(f1s)/len(f1s))
print("ROC AUC Score: ", sum(rocs)/len(rocs))

ZOO: 100%|██████████| 500/500 [00:13<00:00, 36.28it/s]


====================> Model:  RandomForestClassifier(random_state=42)
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75       500
         1.0       0.99      0.34      0.51       500

    accuracy                           0.67      1000
   macro avg       0.80      0.67      0.63      1000
weighted avg       0.80      0.67      0.63      1000

Accuracy:  0.672
Detection Rate:  0.346
F1 Score:  0.5227606461086637
ROC AUC Score:  0.6739999999999999


ZOO: 100%|██████████| 500/500 [00:01<00:00, 271.59it/s]


====================> Model:  MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=32)
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.60      0.99      0.75       500
         1.0       0.98      0.34      0.50       500

    accuracy                           0.67      1000
   macro avg       0.79      0.67      0.63      1000
weighted avg       0.79      0.67      0.63      1000

Accuracy:  0.67
Detection Rate:  0.312
F1 Score:  0.4939759036144578
ROC AUC Score:  0.661


ZOO: 100%|██████████| 500/500 [00:01<00:00, 258.60it/s]


====================> Model:  GaussianNB()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       500
         1.0       0.99      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

Accuracy:  0.989
Detection Rate:  0.99
F1 Score:  0.9939879759519038
ROC AUC Score:  0.991


ZOO: 100%|██████████| 500/500 [00:01<00:00, 283.43it/s]


====================> Model:  DecisionTreeClassifier()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90       500
         1.0       0.99      0.78      0.88       500

    accuracy                           0.89      1000
   macro avg       0.91      0.89      0.89      1000
weighted avg       0.91      0.89      0.89      1000

Accuracy:  0.89
Detection Rate:  0.776
F1 Score:  0.8824833702882483
ROC AUC Score:  0.898


ZOO: 100%|██████████| 500/500 [00:02<00:00, 167.80it/s]


====================> Model:  LogisticRegression()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.77      0.99      0.87       500
         1.0       0.99      0.70      0.82       500

    accuracy                           0.85      1000
   macro avg       0.88      0.85      0.84      1000
weighted avg       0.88      0.85      0.84      1000

Accuracy:  0.844
Detection Rate:  0.696
F1 Score:  0.8243559718969555
ROC AUC Score:  0.848
Accuracy:  0.8130000000000001
Detection Rate:  0.624
F1 Score:  0.7435127735720458
ROC AUC Score:  0.8144
